<a href="https://colab.research.google.com/github/Mrcold2002/colab_code/blob/main/Softmax%E5%9B%9E%E5%BD%92.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
%matplotlib inline
import d2lzh as d2l
from mxnet import autograd,nd

1 读取数据集
  使用Fashion-MNIST数据集，设置批量大小256

In [7]:
batch_size=256
train_iter,test_iter=d2l.load_data_fashion_mnist(batch_size)

2 初始化模型参数

In [24]:
num_inputs=784 # 每个样本大小为28*28=784
num_outputs=19 #总共有10类

w=nd.random.normal(scale=0.01,shape=(num_inputs,num_outputs))
b=nd.zeros(num_outputs)

w.attach_grad()
b.attach_grad()

3 实现softmax运算

In [11]:
# 对同一列axis=0或同一行axis=1的元素求和
# 并在结果中保留行和列这两个维度 keepdims=True
X = nd.array([[1,2,3],[4,5,6]])
X.sum(axis=0,keepdims=True),X.sum(axis=1,keepdims=True)

(
 [[5. 7. 9.]]
 <NDArray 1x3 @cpu(0)>, 
 [[ 6.]
  [15.]]
 <NDArray 2x1 @cpu(0)>)

In [13]:
def softmax(X):
  #矩阵X的行数是样本数，列数是输出个数
  X_exp=X.exp()# 对所有数取exp
  partition=X_exp.sum(axis=1,keepdims=True) # 每一行进行累和
  return X_exp/partition

In [14]:
X=nd.random.normal(shape=(2,5))
X_prob=softmax(X)
X_prob,X_prob.sum(axis=1)
#可以看到对于随机输入，每个元素变为非负数，且和为1

(
 [[0.3800092  0.09019671 0.11808793 0.13589936 0.27580675]
  [0.09572445 0.05599073 0.20952241 0.58733696 0.05142545]]
 <NDArray 2x5 @cpu(0)>, 
 [1. 1.]
 <NDArray 2 @cpu(0)>)

4 定义模型

In [16]:
def net(X):
  return softmax(nd.dot(X.reshape((-1,num_inputs)),w)+b)
  # 通过reshape函数将每张原始图片转为长度为num_inputs向量，而w为（num_inputs,num_outputs）的向量

5 定义损失函数

In [15]:
y_hat=nd.array([[0.1,0.3,0.6],[0.3,0.2,0.5]])
y=nd.array([0,2],dtype='int32')
nd.pick(y_hat,y)
# nd.pick相当于根据y从y_hat中取出对应元素


[0.1 0.5]
<NDArray 2 @cpu(0)>

In [18]:
def cross_entropy(y_hat,y):
  return -nd.pick(y_hat,y).log

6 计算分类准确率

In [20]:
# y_hat.argmax(axis=1) 返回矩阵y_hat每行中的最大值
def accuracy(y_hat,y):
  # 将y_hat中概率最大的类别与实际标签y比较
  return (y_hat.argmax(axis=1)==y.astype('float32')).mean().asscalar()

accuracy(y_hat,y)

0.5

In [25]:
def evaluate_accuracy(data_iter,net):
  acc_sum,n=0.0,0
  for X,y in data_iter:
    y=y.astype('float32')
    acc_sum+=(net(X).argmax(axis=1)==y).sum().asscalar()
    # 在深度学习框架MXNet中，表示对一个NDArray（N维数组）中的所有元素求和，并将结果转换为标量（scalar）。
    # 比较回归后的结果是否正确
    n+=y.size
  return acc_sum/n
# 初始状态，正确率应接近0.1
evaluate_accuracy(test_iter,net)

0.0823